# Egenverdier og evenvektorer (MIP 10.8)

In [1]:
import sympy as sp
import numpy as np

In [2]:
np.sqrt(np.array([-1]).astype('complex'))

array([0.+1.j])

## Python kode for nullrom fra tidligere

In [3]:
import numpy as np

def normer_forste_element(vektor):
    """
    Normaliserer en vektor ved å dele alle elementer på det første ikke-null elementet.
    
    Parametere:
        vektor (np.ndarray): En 1D Numpy-array som skal normaliseres.
    
    Returnerer:
        np.ndarray: En normalisert vektor der det første ikke-null elementet er 1.
    """
    # Finn indeksen til det første elementet i vektoren som ikke er null
    forste_ikke_null_indeks = np.argmax(vektor != 0)
    
    # Hent verdien til det første ikke-null elementet
    forste_ikke_null_element = vektor[forste_ikke_null_indeks]
    
    # Returner den normaliserte vektoren
    return vektor / forste_ikke_null_element

def gauss_jordan(matrise, epsilon=1e-8):
    """
    Utfører Gauss-Jordan eliminasjon på en gitt matrise.
    
    Parametere:
        matrise (np.ndarray): En 2D Numpy-array som representerer matrisen.
        epsilon (float): En liten verdi for å håndtere numerisk presisjon.
    
    Returnerer:
        np.ndarray: En rekkeredusert matrise i redusert trappeform.
    """
    # Sett svært små verdier til null for å unngå numeriske feil
    matrise[np.abs(matrise) < epsilon] = 0
    
    # Hvis matrisen kun består av nuller, returner den uendret
    if np.all(matrise == 0):
        return matrise
    
    # Hvis matrisen har én rad, normaliser den første radens første ikke-null element
    elif len(matrise) == 1:
        matrise[0] = normer_forste_element(matrise[0])
        return matrise
    
    # Beregn radenes summer for å normalisere matrisen (gjør tallene mer håndterbare)
    rad_maksimummer = np.max(np.abs(matrise), axis=1)
    matrise = matrise / rad_maksimummer[rad_maksimummer > epsilon, None]  # Normaliser hver rad
    # Finn kolonner som inneholder ikke-null elementer
    ikke_null_kolonner = np.any(matrise != 0, axis=0)
    
    # Finn indeksen til den første kolonnen med ikke-null elementer
    forste_ikke_null_kolonne = np.argmax(ikke_null_kolonner)
    
    # Finn indeksen til raden med største verdi i den valgte kolonnen (pivot rad)
    pivot_rad_indeks = np.argmax(matrise[:, forste_ikke_null_kolonne])
    
    # Normaliser pivot-raden
    pivot_rad = normer_forste_element(matrise[pivot_rad_indeks])
    
    # Bytt plass på pivot-raden og den første raden
    matrise[pivot_rad_indeks] = matrise[0]
    matrise[0] = pivot_rad
    # Utfør eliminering for å gjøre alle elementene under pivoten null
    matrise[1:] -= (matrise[1:, 0] / matrise[0, forste_ikke_null_kolonne])[:, None] * matrise[0]

    # Kall Gauss-Jordan rekursivt på den nedre delmatrisen
    matrise[1:, 1:] = gauss_jordan(matrise[1:, 1:])
    
    # Gjør den første raden null over pivot-posisjonene til de øvrige radene.
    for rad in matrise[1:]:
        if np.any(rad != 0):  # Hvis raden ikke er null
            # Finn indeksen til det første ikke-null elementet i raden
            forste_ikke_null_kolonne = np.argmax(rad != 0)
            
            # Trekk fra et multiplum av denne raden for å gjøre elementet over pivot null
            matrise[0] -= (matrise[0, forste_ikke_null_kolonne] / rad[forste_ikke_null_kolonne]) * rad
    
    # Returner den resulterende matrisen
    return matrise


In [4]:
def pivot_posisjoner(matrise):
    """
    Finner pivotposisjonene i en rekkeredusert matrise.
    """
    pivot_sett = set()
    pivot_rader = []
    pivot_kolonner = []
    for rad_indeks, rad in enumerate(matrise):
        if np.any(rad != 0):
            første_ikke_null_kolonne = np.argmax(rad != 0)
            pivot_rader.append(rad_indeks)
            pivot_kolonner.append(første_ikke_null_kolonne)
    return pivot_rader, pivot_kolonner

def frie_parametre(matrise):
    """
    Finner bundne og frie parametere i en rekkeredusert matrise.
    """
    _, pivot_kolonner = pivot_posisjoner(matrise)
    alle_kolonner = set(range(matrise.shape[1]))
    return sorted(alle_kolonner.difference(pivot_kolonner))

def null_rom(matrise):
    """
    Finner en basis for nullrommet til en matrise.
    """
    nullrom_basis = []
    redusert_matrise = gauss_jordan(matrise)
    pivot_rader, pivot_kolonner = pivot_posisjoner(redusert_matrise)
    frie = frie_parametre(redusert_matrise)
    
    for fri in frie:
        vektor = np.zeros((matrise.shape[1], 1), dtype=matrise.dtype)
        vektor[fri] = 1
        for rad, kolonne in zip(pivot_rader, pivot_kolonner):
            vektor[kolonne] = -np.sum((redusert_matrise @ vektor)[rad])
        nullrom_basis.append(vektor)
    
    return nullrom_basis

def partikulaer_losning(koeffisientmatrise, høyreside=None):
    """
    Finner en partikulær løsning til ligningssystemet Ax = b.
    """
    if høyreside is None:
        høyreside = np.zeros(koeffisientmatrise.shape[1])
    if len(høyreside.shape) == 1:
        høyreside = høyreside[:, None]
    
    utvidet_matrise = gauss_jordan(np.hstack([koeffisientmatrise, høyreside]))
    radindekser, kolonneindekser = pivot_posisjoner(utvidet_matrise[:, :-1])
    løsning = np.zeros((koeffisientmatrise.shape[1], 1))
    redusert_høyreside = utvidet_matrise[:, -1]
    
    for rad, kolonne in zip(radindekser, kolonneindekser):
        løsning[kolonne] = redusert_høyreside[rad]

    assert np.allclose(koeffisientmatrise @ løsning[None, :], høyreside), "Det finnes ingen løsning til det lineære ligningssystemet"
    return løsning


## Eksempel

In [5]:
A = np.array([
    [0, .5, .5],
    [1, 0, 0],
    [0, .5, .5]
])

In [6]:
np.complex128(12)

np.complex128(12+0j)

In [21]:
def finn_egenvektorer_og_egenverdier(A, dtype='float'):
    assert A.shape[0] == A.shape[1], "matrisen A skal være kvadratisk"

    t = sp.symbols('t')  # Definerer symbolet t, som brukes i det karakteristiske polynomet
    B = A - t * sp.eye(A.shape[0])  # Lager matrisen B = A - t*I, hvor I er identitetsmatrisen
    karakteristisk_polynom = B.det()  # Finner determinant av B, som gir det karakteristiske polynomet
    
    egenverdier = sp.solve(karakteristisk_polynom)  # Løser det karakteristiske polynomet for å finne egenverdiene

    # Lager en liste med tupler av evenverdier, multiplisiteter og egenvektorer
    res = []
    # for ev in sorted(egenverdier, key=lambda x: -np.abs(x)):
    for ev in egenverdier:
        # if np.isreal(np.array(ev).astype(complex)):
        ev = complex(ev)
        if not ev.real == ev:
            print(ev.imag)
            print('kompleks')
        else:
            ev = ev.real
        # ev = np.complex128(ev).astype(dtype)
        A.dtype
        egenvektorer = null_rom((A - ev * np.eye(A.shape[0])).astype(dtype))
        if len(egenvektorer) > 0:
            res.append((ev, len(egenvektorer), egenvektorer))
    
    return res  # Returnerer en liste med tupler av evenverdier, multiplisiteter og egenvektorer

In [22]:
def skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, egenverdier_og_egenvektorer):
    print('Alle vektorer her skal leses som kolonnevektorer\n')
    for key, _, val in sorted(egenverdier_og_egenvektorer, key = lambda x: complex(x[0]).real):
        print('egenverdi:     ', key)
        print('egenvektor:    ', np.array(val[0]).ravel())
        print('A @ evenvektor:', np.array(A @ val[0]).ravel())
        print()

In [29]:
[complex(x[0]).real for x in finn_egenvektorer_og_egenverdier(A)]

-1.0
kompleks
1.0
kompleks


/tmp/nix-shell.EYmWY5/ipykernel_32062/791420158.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  egenvektorer = null_rom((A - ev * np.eye(A.shape[0])).astype(dtype))


[3.0]

In [24]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A))

-1.0
kompleks
1.0
kompleks
Alle vektorer her skal leses som kolonnevektorer

egenverdi:      3.0
egenvektor:     [-2. -3.  1.]
A @ evenvektor: [-6. -9.  3.]



/tmp/nix-shell.EYmWY5/ipykernel_32062/791420158.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  egenvektorer = null_rom((A - ev * np.eye(A.shape[0])).astype(dtype))


Pakken sympy kan regne ut egenverdier og egenvektorer for oss. Uheldigvis regner den ikke helt presis!

In [25]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, sp.Matrix(A).eigenvects())

Alle vektorer her skal leses som kolonnevektorer

egenverdi:      2 - I
egenvektor:     [-1 -1 + I 1]
A @ evenvektor: [-2 + I -1 + 3*I 2 - I]

egenverdi:      2 + I
egenvektor:     [-1 -1 - I 1]
A @ evenvektor: [-2 - I -1 - 3*I 2 + I]

egenverdi:      3
egenvektor:     [-2 -3 1]
A @ evenvektor: [-6 -9 3]



## Hva skjete her?

For 
$$
A = \left[\begin{array}{ccc}
0 & 1/2 & 1/2 \\ 1 & 0 & 0 \\ 0 & 1/2 & 1/2
\end{array}\right]$$
er
$$
A \cdot
\left[\begin{array}{c}
1 \\ -2 \\ 1
\end{array}\right]
= \left[\begin{array}{ccc}
0 & 1/2 & 1/2 \\ 1 & 0 & 0 \\ 0 & 1/2 & 1/2
\end{array}\right]
\left[\begin{array}{c}
1 \\ -2 \\ 1
\end{array}\right]
=
\left[\begin{array}{c}
-1/2 \\ 1 \\ -1/2
\end{array}\right]
= 
-\frac 12 
\left[\begin{array}{c}
1 \\ -2 \\ 1
\end{array}\right]
$$


Vi sier at 
$\left[\begin{array}{c}
1 \\ -2 \\ 1
\end{array}\right]$
er en **egenvektor** til 
$
A = \left[\begin{array}{ccc}
0 & 1/2 & 1/2 \\ 1 & 0 & 0 \\ 0 & 1/2 & 1/2
\end{array}\right]$
med **egenverdi** $-\frac 12$

Tilsvarende er
$\left[\begin{array}{c}
1 \\ -1 \\ 0
\end{array}\right]$
en **egenvektor** til 
$
A = \left[\begin{array}{ccc}
0 & 1/2 & 1/2 \\ 1 & 0 & 0 \\ 0 & 1/2 & 1/2
\end{array}\right]$
med **egenverdi** $0$

og
$\left[\begin{array}{c}
1 \\ 1 \\ 1
\end{array}\right]$
er en **egenvektor** til 
$
A = \left[\begin{array}{ccc}
0 & 1/2 & 1/2 \\ 1 & 0 & 0 \\ 0 & 1/2 & 1/2
\end{array}\right]$
med **egenverdi** $1$

## Metode for å finne egenverdier og egenvektorer

Hvis $A \cdot \vec v = t \vec v$, da er $\vec 0 = A \cdot \vec v - t \vec v = A \cdot \vec v - t I \cdot \vec v = (A - tI)\vec v$. Hvis $\vec v \ne \vec 0$ må vi ha $\det(A - tI) = 0$.

Omvendt, hvis $\det(A - tI) = 0$, da vet vi at det finnes in $\vec v \ne \vec 0$ med $(A - tI) \cdot \vec v = \vec 0$.

Vi ser på $\det(A - tI)$ som en funksjon i den ubestemte $t$, og finner alle nullpunkter for denne funksjonen. Her kan vi for eksempel anvende metoden til Newton. 
Hvis $A$ er en $2 \times 2$ eller $3 \times 3$ matrise da er $\det(A - tI)$ et polynom av grad henholdsvis 2 eller 3. Da kan vi finne nullpunkter, med litt jobbing og litt hell,
finne nullpunkter for hånd.

## Eksempel

In [26]:
A = np.array([
    [2, 1, 1],
    [2, 3, 4],
    [1, -1, 2]
])

In [27]:
finn_egenvektorer_og_egenverdier(A, dtype='float')

-1.0
kompleks
1.0
kompleks


/tmp/nix-shell.EYmWY5/ipykernel_32062/791420158.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  egenvektorer = null_rom((A - ev * np.eye(A.shape[0])).astype(dtype))


[(3.0,
  1,
  [array([[-2.],
          [-3.],
          [ 1.]])])]

In [28]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(A, finn_egenvektorer_og_egenverdier(A, dtype='float'))

-1.0
kompleks
1.0
kompleks
Alle vektorer her skal leses som kolonnevektorer

egenverdi:      3.0
egenvektor:     [-2. -3.  1.]
A @ evenvektor: [-6. -9.  3.]



/tmp/nix-shell.EYmWY5/ipykernel_32062/791420158.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  egenvektorer = null_rom((A - ev * np.eye(A.shape[0])).astype(dtype))


In [ ]:
t = sp.symbols('t')  # Definerer symbolet t, som brukes i det karakteristiske polynomet
B = A - t * sp.eye(A.shape[0])  # Lager matrisen B = A - t*I, hvor I er identitetsmatrisen
karakteristisk_polynom = B.det()  # Finner determinant av B, som gir det karakteristiske polynomet

egenverdier = sp.solve(karakteristisk_polynom)  # Løser det karakteristiske polynomet for å finne egenverdiene
egenverdier, karakteristisk_polynom

Kan du forklare hvorfor vi kun finner en egenvektor?

## Eksempel

In [ ]:
M = np.array([
    [-4, -2, -2],
    [-4, -6, -8],
    [2, 2, 4]
])

In [ ]:
skriv_ut_egenvektorer_og_multiplikasjon_med_matrise(M, finn_egenvektorer_og_egenverdier(M))

In [ ]:
t = sp.symbols('t')  # Definerer symbolet t, som brukes i det karakteristiske polynomet
B = M - t * sp.eye(M.shape[0])  # Lager matrisen B = A - t*I, hvor I er identitetsmatrisen
karakteristisk_polynom = B.det()  # Finner determinant av B, som gir det karakteristiske polynomet

egenverdier = sp.solve(karakteristisk_polynom)  # Løser det karakteristiske polynomet for å finne egenverdiene
egenverdier

In [ ]:
karakteristisk_polynom